# Análise



## Introdução



### Objetivos da Análise

- Visualisar variáveis com maior impacto na demanda e na receita
- Visualizar dias e horários com maior demanda



### Sobre o Dataset

Este dataset contém dados de 1000 pedidos de um restaurante.

## Preparação dos dados

### Carregamento de dados

In [1]:
# Importando a biblioteca Pandas
import pandas as pd

In [2]:
# Carregando dataset
df = pd.read_csv('food_sales.csv')

In [3]:
# Exibição prévia dos dados
df.head()

,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale
0,1,07-03-2022,Aalopuri,Fastfood,20,13,260,NaN,Mr.,Night
1,2,8/23/2022,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon
2,3,11/20/2022,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon
3,4,02-03-2023,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night
4,5,10-02-2022,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening


### Limpeza de dados

In [4]:
# Informações sobre o dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            1000 non-null   int64 
 1   date                1000 non-null   object
 2   item_name           1000 non-null   object
 3   item_type           1000 non-null   object
 4   item_price          1000 non-null   int64 
 5   quantity            1000 non-null   int64 
 6   transaction_amount  1000 non-null   int64 
 7   transaction_type    893 non-null    object
 8   received_by         1000 non-null   object
 9   time_of_sale        1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.3+ KB


**Observações iniciais**:
- Há valores nulos na coluna `transaction_type`, representando mais de 10% dos dados.
- As colunas categóricas estão no formato `object`.
- A coluna `date` possui dados inconsistentes em relação ao formato de data.
- É importante para a análise extrair ano, mês e dia da semana da coluna `date`.

In [5]:
# Substituindo valores nulos pela categoria Unknow
df['transaction_type'] = df['transaction_type'].fillna('Unknow')

In [6]:
# Ajustando variáveis categóricas
category_columns = ['item_name', 'item_type', 'transaction_type', 'received_by', 'time_of_sale']
df[category_columns] = df[category_columns].astype('category')

In [7]:
# Ajustando a variável date para datetime
df['date'] = df['date'].str.replace('/', '-', regex=False)
df['date'] = pd.to_datetime(df['date'], format='%m-%d-%Y')

In [8]:
# Extraindo ano, mês e dia da semana de date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['weekday'] = df['date'].dt.day_name()

In [9]:
# resultado final
df.head()

,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale,year,month,weekday
0,1,2022-07-03,Aalopuri,Fastfood,20,13,260,Unknow,Mr.,Night,2022,July,Sunday
1,2,2022-08-23,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon,2022,August,Tuesday
2,3,2022-11-20,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon,2022,November,Sunday
3,4,2023-02-03,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night,2023,February,Friday
4,5,2022-10-02,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening,2022,October,Sunday


## Análise de produtos

### Desempenho por categoria

Resumo da quantidade vendida e da receita por categoria.

In [ ]:
# Quantidade e receita por categoria
df.groupby('item_type', observed=True)[['quantity', 'transaction_amount']].sum().reset_index()

#### Quantidade vendida

In [ ]:
# Quantidade vendida por categoria
quantity_category = df.groupby('item_type', observed=True)['quantity'].sum()
quantity_category.plot(
    title='Quantidade Vendida',
    kind='pie',
    autopct='%1.1f%%',
    startangle=90,
    ylabel=''
);

#### Receita

In [ ]:
# Receita por categoria
amount_category = df.groupby('item_type', observed=True)['transaction_amount'].sum()
amount_category.plot(
    title='Receita',
    kind='pie',
    autopct='%1.1f%%',
    startangle=90,
    ylabel=''
);

Conclui-se que **Fastfood** é a categoria com maior impacto na quantidade e na receita.

### Desempenho por item

Resumo da quantidade vendida e da receita por item.

In [ ]:
# Quantidade e receita por item
df.groupby('item_name', observed=True)[['quantity', 'transaction_amount']].sum().reset_index()

#### Quantidade vendida

In [ ]:
# Quantidade vendida por item
quantity_item = df.groupby('item_name', observed=True)['quantity'].sum().sort_values()
quantity_item.plot(kind='barh');

#### Receita

In [ ]:
# Receita por item
amount_item = df.groupby('item_name', observed=True)['transaction_amount'].sum().sort_values()
amount_item.plot(kind='barh');

Conclui-se que **Cold coffee** tem maior impacto na quantidade vendida equanto **Sandwich** tem maior impacno na receita.

## Análise de vendas

### Desempenho por vendedor

#### Pedidos

In [ ]:
# Quantidade
order_received = df.groupby('received_by', observed=True)['received_by'].count()
order_received.plot(kind='pie');

#### Receita

In [ ]:
# Receita
amount_received = df.groupby('received_by', observed=True)['transaction_amount'].sum()
amount_received.plot(kind='pie');

## Análise temporal

### Desempenho anual

#### Itens por ano

In [ ]:
# Itens
items = df.groupby('year', observed=True)['quantity'].sum()
items.plot(kind='bar');

#### Pedidos por ano

In [ ]:
# Pedidos
orders_year = df.groupby('year', observed=True)['year'].count()
orders_year.plot(kind='bar');

#### Receita por ano

In [ ]:
# Receita
amount_year = df.groupby('year', observed=True)['transaction_amount'].sum()
amount_year.plot(kind='bar');

### Desempenho mensal

In [ ]:
df_22 = df[df['year'] == 2022]
df_23 = df[df['year'] == 2023]

#### Itens por mês

2022

In [ ]:
# Itens 2022
items_month = df_22.groupby('month', observed=True)['quantity'].sum()
items_month.plot(kind='barh');

2023

In [ ]:
# Itens 2023
items_month = df_23.groupby('month', observed=True)['quantity'].sum()
items_month.plot(kind='barh');

#### Pedidos por ano

In [ ]:
# Pedidos
orders_year = df.groupby('year', observed=True)['year'].count()
orders_year.plot(kind='bar');

#### Receita por ano

In [ ]:
# Receita
amount_year = df.groupby('year', observed=True)['transaction_amount'].sum()
amount_year.plot(kind='bar');

### Receita mensal

In [ ]:
df.head()

In [ ]:
amount = df[df['year'] == 2022]
amount = amount[['date', 'transaction_amount']]

In [ ]:
amount.set_index('date', inplace=True)

In [ ]:
amount_date = amount.resample('ME').sum()

In [ ]:
amount_date

In [ ]:
amount_date.plot(kind='line', xlabel='');